# 1. Goals

Predicting store sales 

# 2. Dataset Description


**train.csv**
The training data, comprising time series of features store_nbr, family, and onpromotion as well as ***the target sales***.
* **store_nbr** identifies the store at which the products are sold.
* **family** identifies the type of product sold.
* **onpromotion** gives the total number of items in a product family that were being promoted at a store at a given date.

* **sales** gives the total sales for a product family at a particular store at a given date. Fractional values are possible since products can be sold in fractional units (1.5 kg of cheese, for instance, as opposed to 1 bag of chips).

**test.csv**
* The test data, having the same features as the training data. You will predict the target sales for the dates in this file.
* The dates in the test data are for the 15 days after the last date in the training data.

**stores.csv**
Store metadata, including:
* **city**
* **state**
* **type**
* **cluster**: a grouping of similar stores.

**transactions.csv**
* date
* store_nbr
* transactions

**oil.csv**
* Daily oil price. Includes values during both the train and test data timeframes. (Ecuador is an oil-dependent country and it's economical health is highly vulnerable to shocks in oil prices.)

**holidays_events.csv**
* Holidays and Events, with metadata
* NOTE: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

**Additional Notes**
* Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. Supermarket sales could be affected by this.
* A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

# 3. Import libraries

In [ ]:
# "magic commands" to enable autoreload of the imported packages
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import re
from pathlib import Path

# Numpy and Pandas
import numpy as np
import pandas as pd
pd.options.display.max_colwidth = 500
pd.options.display.max_columns = 100
pd.options.display.float_format = '{:.2f}'.format
# pd.options.display.precision 


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1. Load data & convert date to datetime


* holidays_events.csv
* oil.csv
* stores.csv
* transactions.csv

* train.csv
* test.csv


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Directory containing the raw data
raw_data_dir = Path("/content/drive/MyDrive/Colab Notebooks/Store-Sales/raw_data")

# Specify data types 
train_dtypes = {
    "id": "category",
    "store_nbr": "category",
    "family": "category",
    "sales": np.float32,
    "onpromotion": np.float32,
}

test_dtypes = {
    "id": "category",
    "store_nbr": "category",
    "family": "category",
    "onpromotion": np.float32,
}
  
stores_dtypes = {
    "store_nbr": "category",
    "city": "category",
    "state": "category",
    "type": "category", 
    "cluster": "category"
}

transactions_dtypes = {
    "store_nbr": "category",
    "transactions": np.float32
}

holidays_events_dtypes = {
    "type": "category",
    "locale": "category",
    "locale_name": "category",
    "description": "category",
    "transferred": "category",
}

oil_dtypes = {
    "dcoilwtico" : np.float32   
}


# Load data
train_df = pd.read_csv(raw_data_dir / "train.csv", dtype=train_dtypes,  parse_dates=["date"])
test_df = pd.read_csv(raw_data_dir / "test.csv", dtype=test_dtypes,  parse_dates=["date"])
stores_df = pd.read_csv(raw_data_dir / "stores.csv", dtype=stores_dtypes)
transactions_df = pd.read_csv(raw_data_dir / "transactions.csv",  dtype=transactions_dtypes, parse_dates=["date"])
holidays_events_df = pd.read_csv(raw_data_dir / "holidays_events.csv", dtype=holidays_events_dtypes,  parse_dates=["date"])
oil_df = pd.read_csv(raw_data_dir / "oil.csv", dtype=oil_dtypes,  parse_dates=["date"])

sample_submission_df = pd.read_csv(raw_data_dir / "sample_submission.csv")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
print(test_df.date.iloc[28511] - test_df.date.iloc[0])
test_df

15 days 00:00:00


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0.00
1,3000889,2017-08-16,1,BABY CARE,0.00
2,3000890,2017-08-16,1,BEAUTY,2.00
3,3000891,2017-08-16,1,BEVERAGES,20.00
4,3000892,2017-08-16,1,BOOKS,0.00
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1.00
28508,3029396,2017-08-31,9,PREPARED FOODS,0.00
28509,3029397,2017-08-31,9,PRODUCE,1.00
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9.00


In [ ]:
dfs = {"train_df": train_df, "test_df": test_df, "stores_df": stores_df, "transactions_df": transactions_df, "holidays_events_df": holidays_events_df, "oil_df": oil_df}

for key, df in dfs.items(): 
  print(f"----------{key}----------")
  print(df.info(), df.describe(), "\n")

----------train_df----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000888 entries, 0 to 3000887
Data columns (total 6 columns):
 #   Column       Dtype         
---  ------       -----         
 0   id           category      
 1   date         datetime64[ns]
 2   store_nbr    category      
 3   family       category      
 4   sales        float32       
 5   onpromotion  float32       
dtypes: category(3), datetime64[ns](1), float32(2)
memory usage: 150.4 MB
None            sales  onpromotion
count 3000888.00   3000888.00
mean      357.78         2.60
std      1102.00        12.22
min         0.00         0.00
25%         0.00         0.00
50%        11.00         0.00
75%       195.85         0.00
max    124717.00       741.00 

----------test_df----------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28512 entries, 0 to 28511
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   id   

<ipython-input-4-c348b6b4a573>:5: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  print(df.info(), df.describe(), "\n")


In [ ]:
## Add log of "sales" (Target) to train_df
# # Take log of "sales"
# # Because we have sales=0 on some days, so we use np.log1p
# train_df["sales_log"] = np.log1p(train_df["sales"]+1)

# # Change the order of the columns 
# train_df = train_df.loc[:, ["id", "date", "store_nbr", "family", "sales", "sales_log", "onpromotion"]]
# train_df.head()

# 2. Merge the datasets

We will create the base dataset for feature engineering and modelling. The base dataset is the data after combining all the relevant information we need for feature engineering and modelling. 

## Merge train_df with transactions_df

In [ ]:
transactions_df.sort_values(by=["store_nbr", "date"])

,date,store_nbr,transactions
1,2013-01-02,1,2111.00
47,2013-01-03,1,1833.00
93,2013-01-04,1,1863.00
139,2013-01-05,1,1509.00
185,2013-01-06,1,520.00
...,...,...,...
83226,2017-08-11,9,1848.00
83280,2017-08-12,9,1920.00
83334,2017-08-13,9,1745.00
83388,2017-08-14,9,1971.00


In [ ]:
# Merge
train_transactions_df = train_df.merge(transactions_df, how="left", on=["date", "store_nbr"])

# NOTE: each value in the "transactions" column is the number of transactions per store for a given date
train_transactions_df[train_transactions_df["date"] == "2013-01-02"]

,id,date,store_nbr,family,sales,onpromotion,transactions
1782,1782,2013-01-02,1,AUTOMOTIVE,2.00,0.00,2111.00
1783,1783,2013-01-02,1,BABY CARE,0.00,0.00,2111.00
1784,1784,2013-01-02,1,BEAUTY,2.00,0.00,2111.00
1785,1785,2013-01-02,1,BEVERAGES,1091.00,0.00,2111.00
1786,1786,2013-01-02,1,BOOKS,0.00,0.00,2111.00
...,...,...,...,...,...,...,...
3559,3559,2013-01-02,9,POULTRY,651.29,0.00,2940.00
3560,3560,2013-01-02,9,PREPARED FOODS,83.00,0.00,2940.00
3561,3561,2013-01-02,9,PRODUCE,0.00,0.00,2940.00
3562,3562,2013-01-02,9,SCHOOL AND OFFICE SUPPLIES,0.00,0.00,2940.00


In [ ]:
# Check sales vs. transactions
check_ = train_transactions_df.groupby(["store_nbr", "date"])[["sales", "transactions"]].mean()
idx = pd.IndexSlice
check_.loc[idx[:, "2013-01-02"], :]
# check_["compare"] = np.where((check_["sales"] == check_["transactions"]), "yes", "no")
# check_
# check_[check_["compare"]=="yes"]

,,sales,transactions
store_nbr,date,,
1,2013-01-02,224.76,2111.00
10,2013-01-02,164.55,1293.00
11,2013-01-02,555.44,3547.00
12,2013-01-02,177.41,1362.00
13,2013-01-02,167.18,1102.00
14,2013-01-02,233.28,2002.00
15,2013-01-02,199.33,1622.00
16,2013-01-02,189.17,1167.00
17,2013-01-02,232.53,1580.00


In [ ]:
# Fill NaN in "transaction" 
print(train_transactions_df["transactions"].isna().value_counts())
train_transactions_df["transactions"] = train_transactions_df["transactions"].fillna(0)
train_transactions_df

False    2755104
True      245784
Name: transactions, dtype: int64


,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0.00,0.00
...,...,...,...,...,...,...,...
3000883,3000883,2017-08-15,9,POULTRY,438.13,0.00,2155.00
3000884,3000884,2017-08-15,9,PREPARED FOODS,154.55,1.00,2155.00
3000885,3000885,2017-08-15,9,PRODUCE,2419.73,148.00,2155.00
3000886,3000886,2017-08-15,9,SCHOOL AND OFFICE SUPPLIES,121.00,8.00,2155.00


## Combine train_transactions_df and test_df 

In [ ]:
# Check last date of train_df and first date of test_df
display(train_transactions_df.tail(1), test_df.head(1))

# Combine train_df and test_df 
train_test_transactions_df = pd.concat([train_transactions_df, test_df])
print(train_test_transactions_df.shape)
train_test_transactions_df.head()

,id,date,store_nbr,family,sales,onpromotion,transactions
3000887,3000887,2017-08-15,9,SEAFOOD,16.00,0.00,2155.00


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0.00


(3029400, 7)


,id,date,store_nbr,family,sales,onpromotion,transactions
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,0.00
1,1,2013-01-01,1,BABY CARE,0.00,0.00,0.00
2,2,2013-01-01,1,BEAUTY,0.00,0.00,0.00
3,3,2013-01-01,1,BEVERAGES,0.00,0.00,0.00
4,4,2013-01-01,1,BOOKS,0.00,0.00,0.00


## Merge with stores_df

In [ ]:
train_test_transactions_stores_df = train_test_transactions_df.merge(stores_df, how="left", on=["store_nbr"])
print(train_test_transactions_stores_df.shape)
train_test_transactions_stores_df.head()

(3029400, 11)


,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,0.00,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.00,0.00,0.00,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.00,0.00,0.00,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.00,0.00,0.00,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.00,0.00,0.00,Quito,Pichincha,D,13


## Merge with local, regional, national holidays 

NOTE: Commented cells are for checking, can be commented out if needed

#### Step 1: Create holiday_df (from holidays_events_df)
* **type-Transfer**: Pay special attention to the transferred column. A holiday that is transferred officially falls on that calendar day, but was moved to another date by the government. A transferred day is more like a normal day than a holiday. To find the day that it was actually celebrated, look for the corresponding row where type is Transfer. 
    * For example, the holiday Independencia de Guayaquil was transferred from 2012-10-09 to 2012-10-12, which means it was celebrated on 2012-10-12. 
   
*  **type-Bridge & type-Work Day**:  Days that are type Bridge are extra days that are added to a holiday (e.g., to extend the break across a long weekend). These are frequently made up by the type Work Day which is a day not normally scheduled for work (e.g., Saturday) that is meant to payback the Bridge.
* **type-Additional**: Additional holidays are days added a regular calendar holiday, for example, as typically happens around Christmas (making Christmas Eve a holiday).

In [ ]:
# # Check holiday types
# print(holidays_events_df.shape)
# print(holidays_events_df["type"].value_counts())

# # Check locale types
# print(holidays_events_df.shape)
# print(holidays_events_df["locale"].value_counts())

# # There are 12 transferred holidays (meaning that these days are no longer holidays)
# # So, we have 12 actual holidays (corresponding to the 12 transferred holidays )
# # This means that we do not need to keep rows where transferred == True because they are not holidays
# display(holidays_events_df[holidays_events_df["transferred"]=="True"], holidays_events_df[holidays_events_df["type"]=="Transfer"])

In [ ]:
# Create holiday_df

# Keep rows having "transferred" == False only (338 days)
holiday_df = holidays_events_df[(holidays_events_df["transferred"]=="False")]

# Drop "description" and "transferred" columns 
holiday_df = holiday_df.drop(["description", "transferred"], axis=1)

# Drop the duplicated value on the 2016-05-08
holiday_df = holiday_df.drop_duplicates(keep='first')

# Change "type" column name:
holiday_df = holiday_df.rename(columns = {"type": "holiday_type"})

#### Step 2: Create separate local, regional, national dataframes 

In [ ]:
# # Some areas/cities celebrate holidays on the same date

# print(holiday_df["date"].duplicated().value_counts()) 
# holiday_df[holiday_df["date"].duplicated(keep=False)]

# # local_holiday = holiday_df[holiday_df.locale == "Local"]
# # print(local_holiday["date"].duplicated().value_counts()) 
# # local_holiday[local_holiday["date"].duplicated(keep=False)]

In [ ]:
# # Compare "city" and "state" from stores_df and "locale_name" from holiday_events_df

# ###############
# # From stores_df
# list_city = np.sort(stores_df.city.unique())   
# print(f"the number of cities from stores_df is {len(list_city)}")
# print(list_city)

# list_state = np.sort(stores_df.state.unique())  
# print("----")
# print(f"the number of states from stores_df is {len(list_state)}")
# print(list_state)

# ###############
# # From holiday_df
# list_locale_name = np.sort(holiday_df.locale_name.unique())
# print("----")
# print(f"the number of locale_names from holiday_df is {len(list_locale_name)}")
# print(list_locale_name)

# ###############
# # Check common places
# common_city_locale = sorted(list(set(list_city).intersection(list_locale_name)))
# print("----")
# print(f"the number of common places from city and locale_name is {len(common_city_locale)}")
# print(common_city_locale) 

# common_state_locale = sorted(list(set(list_state).intersection(list_locale_name)))
# print("----")
# print(f"the number of common places from state and locale_name is {len(common_state_locale)}")
# print(common_state_locale) 


Local holidays only take place in some cities (and thus for some stores) not for all stores.

Thus, if we merge the holiday_df with train_test_stores_df on "date", we will get incorrect information on Local holidays for the cities. Check: pd.merge(train_test_stores_df, holiday_df, how="left", on="date") 

We create separate 3 dataframes for Local, Regional, and National holidays and merge them one by one with train_test_stores_df. 3 dataframes are named as **local, regional, national**

##### local

In [ ]:
# LOCAL
local = holiday_df[holiday_df["locale"]=="Local"]
local = local.rename(columns={"holiday_type": "Local_holiday_type", "locale": "is_Local_holiday", "locale_name": "Local_locale_name"})
display(local.shape, local.Local_holiday_type.value_counts())

(148, 4)

Holiday       133
Additional     11
Transfer        4
Bridge          0
Event           0
Work Day        0
Name: Local_holiday_type, dtype: int64

In [ ]:
# LOCAL
# Since there are only 4 Transfer which are also official holidays, we collapse them with "Holiday" category. We also remove holiday_type categories with no values
local["Local_holiday_type"] = local["Local_holiday_type"].replace(to_replace=["Transfer"], value="Holiday")
local["Local_holiday_type"] = local["Local_holiday_type"].cat.remove_unused_categories()

# Check and drop duplicates
drop_index = local[local.loc[:,["date", "Local_locale_name"]].duplicated(keep='first')].index
local = local.drop(index=drop_index)
print("Duplicated values:", local.loc[:,["date", "Local_locale_name"]].duplicated(keep='first').sum()) 

display(local.shape, local.Local_holiday_type.value_counts())
# local.head()

Duplicated values: 0


(147, 4)

Holiday       136
Additional     11
Name: Local_holiday_type, dtype: int64

##### regional

In [ ]:
# REGIONAL 
regional = holiday_df[holiday_df["locale"]=="Regional"]
regional = regional.rename(columns={"holiday_type": "Regional_holiday_type", "locale": "is_Regional_holiday", "locale_name": "Regional_locale_name"})
display(regional.shape, regional.Regional_holiday_type.value_counts())

(24, 4)

Holiday       24
Additional     0
Bridge         0
Event          0
Transfer       0
Work Day       0
Name: Regional_holiday_type, dtype: int64

In [ ]:
# REGIONAL
# We remove holiday_type categories with no values
regional["Regional_holiday_type"] = regional["Regional_holiday_type"].cat.remove_unused_categories()

display(regional.shape, regional.Regional_holiday_type.value_counts())

regional.head()

(24, 4)

Holiday    24
Name: Regional_holiday_type, dtype: int64

,date,Regional_holiday_type,is_Regional_holiday,Regional_locale_name
1,2012-04-01,Holiday,Regional,Cotopaxi
7,2012-06-25,Holiday,Regional,Imbabura
23,2012-11-06,Holiday,Regional,Santo Domingo de los Tsachilas
24,2012-11-07,Holiday,Regional,Santa Elena
47,2013-04-01,Holiday,Regional,Cotopaxi


##### national

In [ ]:
# NATIONAL 
national = holiday_df[holiday_df["locale"]=="National"]
national = national.rename(columns={"holiday_type": "National_holiday_type", "locale": "is_National_holiday", "locale_name": "National_locale_name"})
display(national.shape, national.National_holiday_type.value_counts())

(165, 4)

Event         55
Holiday       52
Additional    40
Transfer       8
Bridge         5
Work Day       5
Name: National_holiday_type, dtype: int64

In [ ]:
# NATIONAL
# There are 3 duplicated dates between "Additional" & "Bridge". 
_ = national[(national["National_holiday_type"] == "Additional") | (national["National_holiday_type"] == "Bridge")]
bool_series = _.duplicated(subset="date", keep=False)
_[bool_series]

# Since both "Additional" & "Bridge" are extra days, we can combine them into one category and then drop the duplicates. 
# Additionally, there are only 8 "Transfer" which are also official holidays. So we collapse them with "Holiday" category
national["National_holiday_type"] = national["National_holiday_type"].replace(to_replace=["Bridge", "Transfer"], value=["Additional", "Holiday"])
national = national.drop_duplicates(keep='first')

# 2016-05-01 and 2016-05-07 have duplicates. We also drop them
# national[national.date.duplicated(keep=False)]
drop_index = national[national.date.duplicated(keep='first')].index
national = national.drop(index=drop_index)
print("Duplicated values:", national.date.duplicated(keep='first').sum()) 

# Remove "Work Day" rows and category
national = national.drop(national[national.National_holiday_type == "Work Day"].index)
national["National_holiday_type"] = national["National_holiday_type"].cat.remove_categories("Work Day")

# Set order for holiday_type categories 
national["National_holiday_type"] = national["National_holiday_type"].cat.reorder_categories(["Holiday", "Additional", "Event"], ordered=True)

# Count values of the National holiday type
display(national.shape, national.National_holiday_type.value_counts()) 

national.head()

Duplicated values: 0


(155, 4)

Holiday       60
Event         53
Additional    42
Name: National_holiday_type, dtype: int64

,date,National_holiday_type,is_National_holiday,National_locale_name
14,2012-08-10,Holiday,National,Ecuador
20,2012-10-12,Holiday,National,Ecuador
21,2012-11-02,Holiday,National,Ecuador
22,2012-11-03,Holiday,National,Ecuador
31,2012-12-21,Additional,National,Ecuador


### Step 3: Merge train_test_transactions_stores_df with local, regional, and national


In [ ]:
train_test_transactions_stores_holidays_df = train_test_transactions_stores_df.merge(local, how="left", left_on=["date", "city"], right_on=["date", "Local_locale_name"])
train_test_transactions_stores_holidays_df = train_test_transactions_stores_holidays_df.merge(regional, how="left", left_on=["date", "state"], right_on=["date", "Regional_locale_name"])
train_test_transactions_stores_holidays_df = train_test_transactions_stores_holidays_df.merge(national, how="left", on="date")

In [ ]:
# Check whether "city" and "..._local_name" have same values
check_ = train_test_transactions_stores_holidays_df[train_test_transactions_stores_holidays_df['city'] == 'Quito']
check_ = check_[check_["is_Local_holiday"] == "Local"]
check_

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster,Local_holiday_type,is_Local_holiday,Local_locale_name,Regional_holiday_type,is_Regional_holiday,Regional_locale_name,National_holiday_type,is_National_holiday,National_locale_name
602316,602316,2013-12-05,1,AUTOMOTIVE,0.00,0.00,1930.00,Quito,Pichincha,D,13,Additional,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
602317,602317,2013-12-05,1,BABY CARE,0.00,0.00,1930.00,Quito,Pichincha,D,13,Additional,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
602318,602318,2013-12-05,1,BEAUTY,2.00,0.00,1930.00,Quito,Pichincha,D,13,Additional,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
602319,602319,2013-12-05,1,BEVERAGES,1008.00,0.00,1930.00,Quito,Pichincha,D,13,Additional,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
602320,602320,2013-12-05,1,BOOKS,0.00,0.00,1930.00,Quito,Pichincha,D,13,Additional,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2553601,2553601,2016-12-06,9,POULTRY,618.00,2.00,2447.00,Quito,Pichincha,B,6,Holiday,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
2553602,2553602,2016-12-06,9,PREPARED FOODS,113.71,1.00,2447.00,Quito,Pichincha,B,6,Holiday,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
2553603,2553603,2016-12-06,9,PRODUCE,2518.06,135.00,2447.00,Quito,Pichincha,B,6,Holiday,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN
2553604,2553604,2016-12-06,9,SCHOOL AND OFFICE SUPPLIES,2.00,0.00,2447.00,Quito,Pichincha,B,6,Holiday,Local,Quito,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# Check whether "city" and "..._local_name" have same values
check_ = train_test_transactions_stores_holidays_df[train_test_transactions_stores_holidays_df['state'] == 'Cotopaxi']
check_ = check_[check_["is_Regional_holiday"] == "Regional"]
check_

,id,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster,Local_holiday_type,is_Local_holiday,Local_locale_name,Regional_holiday_type,is_Regional_holiday,Regional_locale_name,National_holiday_type,is_National_holiday,National_locale_name
160479,160479,2013-04-01,12,AUTOMOTIVE,3.00,0.00,1313.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
160480,160480,2013-04-01,12,BABY CARE,0.00,0.00,1313.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
160481,160481,2013-04-01,12,BEAUTY,4.00,0.00,1313.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
160482,160482,2013-04-01,12,BEVERAGES,762.00,0.00,1313.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
160483,160483,2013-04-01,12,BOOKS,0.00,0.00,1313.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2756914,2756914,2017-04-01,13,POULTRY,197.73,0.00,1183.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
2756915,2756915,2017-04-01,13,PREPARED FOODS,40.00,0.00,1183.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
2756916,2756916,2017-04-01,13,PRODUCE,296.78,0.00,1183.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN
2756917,2756917,2017-04-01,13,SCHOOL AND OFFICE SUPPLIES,0.00,0.00,1183.00,Latacunga,Cotopaxi,C,15,NaN,NaN,NaN,Holiday,Regional,Cotopaxi,NaN,NaN,NaN


## Merge with oil_df 

In [ ]:
train_test_transactions_stores_holidays_oil_df = train_test_transactions_stores_holidays_df.merge(oil_df, how="left", on="date")

# 4. Prepare the base dataset

In [ ]:
df = train_test_transactions_stores_holidays_oil_df.copy()

# Check shape, columns and datatypes
display(df.shape, df.columns, df.dtypes)

(3029400, 21)

Index(['id', 'date', 'store_nbr', 'family', 'sales', 'onpromotion',
       'transactions', 'city', 'state', 'type', 'cluster',
       'Local_holiday_type', 'is_Local_holiday', 'Local_locale_name',
       'Regional_holiday_type', 'is_Regional_holiday', 'Regional_locale_name',
       'National_holiday_type', 'is_National_holiday', 'National_locale_name',
       'dcoilwtico'],
      dtype='object')

id                               object
date                     datetime64[ns]
store_nbr                      category
family                         category
sales                           float32
onpromotion                     float32
transactions                    float32
city                             object
state                            object
type                           category
cluster                        category
Local_holiday_type             category
is_Local_holiday               category
Local_locale_name              category
Regional_holiday_type          category
is_Regional_holiday            category
Regional_locale_name           category
National_holiday_type          category
is_National_holiday            category
National_locale_name           category
dcoilwtico                      float32
dtype: object

### Check duplicates and missing values

In [ ]:
# CHECK DUPLICATES 
df_check = df.set_index(["family", "store_nbr", "date"]).sort_index()
print("Duplicated values: ", df_check.index.duplicated(keep=False).sum(), "\n")
del df_check

# CHECK MISSING VALUES 
print("Missing values: ") 
df.isna().sum() 

# NOTE:
# "sales": 28512 missing values are from the test_df (test_df.shape = (28512, 5))

Duplicated values:  0 

Missing values: 


id                             0
date                           0
store_nbr                      0
family                         0
sales                      28512
onpromotion                    0
transactions               28512
city                           0
state                          0
type                           0
cluster                        0
Local_holiday_type       3017454
is_Local_holiday         3017454
Local_locale_name        3017454
Regional_holiday_type    3028377
is_Regional_holiday      3028377
Regional_locale_name     3028377
National_holiday_type    2795958
is_National_holiday      2795958
National_locale_name     2795958
dcoilwtico                935550
dtype: int64

In [ ]:
# DEAL WITH MISSING VALUES: 
df["oil_price_interpolated"] = df["dcoilwtico"].interpolate(limit_direction='both')
    # .interpolate: Fill NaN values using an interpolation method (Here, we choose the default method='linear')

# REORDER THE CATEGORIES OF HOLIDAY FEATURES
df["Local_holiday_type"] = df["Local_holiday_type"].cat.add_categories(["no_holiday"]).fillna("no_holiday")
df["Local_holiday_type"] = df["Local_holiday_type"].cat.reorder_categories(["no_holiday", "Holiday", "Additional"], ordered=True)

df["Regional_holiday_type"] = df["Regional_holiday_type"].cat.add_categories(["no_holiday"]).fillna("no_holiday")
df["Regional_holiday_type"] = df["Regional_holiday_type"].cat.reorder_categories(["no_holiday", "Holiday"], ordered=True)

df["National_holiday_type"] = df["National_holiday_type"].cat.add_categories(["no_holiday"]).fillna("no_holiday")
df["National_holiday_type"] = df["National_holiday_type"].cat.reorder_categories(["no_holiday", "Holiday", "Additional", "Event"], ordered=True)

df["is_Local_holiday"] = df["is_Local_holiday"].cat.add_categories(["no_holiday"]).fillna("no_holiday")
df["is_Regional_holiday"] = df["is_Regional_holiday"].cat.add_categories(["no_holiday"]).fillna("no_holiday")
df["is_National_holiday"] = df["is_National_holiday"].cat.add_categories(["no_holiday"]).fillna("no_holiday")


# DROP UNNECCESSARY COLUMNS 
# Drop "id", "..._locale_name" and 'dcoilwtico' columns 
df = df.drop(["id", "Local_locale_name", "Regional_locale_name", "National_locale_name", "dcoilwtico"], axis=1)

# CHECK MISSING VALUES AGAIN
df.isna().sum()

date                          0
store_nbr                     0
family                        0
sales                     28512
onpromotion                   0
transactions              28512
city                          0
state                         0
type                          0
cluster                       0
Local_holiday_type            0
is_Local_holiday              0
Regional_holiday_type         0
is_Regional_holiday           0
National_holiday_type         0
is_National_holiday           0
oil_price_interpolated        0
dtype: int64

### Add earthquake and wage_day features

In [ ]:
# ADD EARTHQUAKE FEATURES
earthquake_day = ['yes' if x.strftime('%Y-%m-%d') == '2016-04-16' else 'no' for x in df['date']]
earthquake_impact = ['yes' if (x.strftime('%Y-%m-%d') > '2016-04-16') & (x.strftime('%Y-%m-%d') < '2016-05-16') else 'no' for x in df['date']]   # one month

df['earthquake_day'] = earthquake_day
df['earthquake_day'] = df['earthquake_day'].astype(dtype="category")
df['earthquake_impact'] = earthquake_impact
df['earthquake_impact'] = df['earthquake_impact'].astype(dtype="category")

#A magnitude 7.8 earthquake struck Ecuador on April 16, 2016. 
#People rallied in relief efforts donating water and other first need products which greatly affected supermarket sales for several weeks after the earthquake.

In [ ]:
# ADD WAGE DAY FEATURE
first_days = df[df["date"].dt.strftime('%Y-%m-%d').str.contains(r"-01$", regex=True)]    #search for the 1st of one month, and then use index-1 to get the last day of the month before
fifteenth_days = df[df["date"].dt.strftime('%Y-%m-%d').str.contains(r"-15$", regex=True)]

first_days_index = first_days.index
last_days_index = first_days_index[1:] - 1
fifteenth_days_index = fifteenth_days.index
# print(first_days_index, '\n', last_days_index, '\n', fifteenth_days_index)

df['wage_day'] = "no" 
df.loc[last_days_index, 'wage_day'] = "yes"
df.loc[fifteenth_days_index, 'wage_day'] = "yes"
df['wage_day'] = df['wage_day'].astype(dtype="category")

#Wages in the public sector are paid every two weeks on the 15 th and on the last day of the month. 
#Supermarket sales could be affected by this

### Check the final base dataset

In [ ]:
# Change data types for "city" and "state"
df["city"] = df["city"].astype(dtype="category")                  
df["state"] = df["state"].astype(dtype="category")   

# Change "BREAD/BAKERY" to "BREAD BAKERY"
df["family"] = df["family"].replace(to_replace=["BREAD/BAKERY"], value="BREAD BAKERY") 

# Check df 
df.head()

,date,store_nbr,family,sales,onpromotion,transactions,city,state,type,cluster,Local_holiday_type,is_Local_holiday,Regional_holiday_type,is_Regional_holiday,National_holiday_type,is_National_holiday,oil_price_interpolated,earthquake_day,earthquake_impact,wage_day
0,2013-01-01,1,AUTOMOTIVE,0.00,0.00,0.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,Holiday,National,93.14,no,no,yes
1,2013-01-01,1,BABY CARE,0.00,0.00,0.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,Holiday,National,93.14,no,no,yes
2,2013-01-01,1,BEAUTY,0.00,0.00,0.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,Holiday,National,93.14,no,no,yes
3,2013-01-01,1,BEVERAGES,0.00,0.00,0.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,Holiday,National,93.14,no,no,yes
4,2013-01-01,1,BOOKS,0.00,0.00,0.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,Holiday,National,93.14,no,no,yes


In [ ]:
# Check data types
df.dtypes

date                      datetime64[ns]
store_nbr                       category
family                          category
sales                            float32
onpromotion                      float32
transactions                     float32
city                            category
state                           category
type                            category
cluster                         category
Local_holiday_type              category
is_Local_holiday                category
Regional_holiday_type           category
is_Regional_holiday             category
National_holiday_type           category
is_National_holiday             category
oil_price_interpolated           float32
earthquake_day                  category
earthquake_impact               category
wage_day                        category
dtype: object

# 5. Save the base dataset as parquet file

In [ ]:
# Set directory
base_data_dir = Path("/content/drive/MyDrive/Colab Notebooks/Store-Sales")


# If exist_ok is false (the default), FileExistsError is raised if the target directory already exists.
# If exist_ok is true, FileExistsError exceptions will be ignored (same behavior as the POSIX mkdir -p command), but only if the last path component is not an existing non-directory file.

# Save the base dataset
# We partition the data by "family" so we can make it easier to read the data 
df.to_parquet(path=base_data_dir, 
              index=False, 
              engine="pyarrow",
              partition_cols=["family"]
              )

In [ ]:
# Check: Load a subset of the base dataset

df_check = pd.read_parquet(
    path=base_data_dir, 
    engine="pyarrow",
    filters=[("family", "=", "AUTOMOTIVE"), # use filters to load a subset of the base dataset, 2017-08-15 is the last date of train_df
             ("date", "<=", pd.to_datetime("2017-08-15"))]
)      

df_check.dtypes

date                      datetime64[ns]
store_nbr                       category
sales                            float32
onpromotion                      float32
transactions                     float32
city                            category
state                           category
type                            category
cluster                         category
Local_holiday_type              category
is_Local_holiday                category
Regional_holiday_type           category
is_Regional_holiday             category
National_holiday_type           category
is_National_holiday             category
oil_price_interpolated           float32
earthquake_day                  category
earthquake_impact               category
wage_day                        category
family                          category
dtype: object

In [ ]:
df_check

,date,store_nbr,sales,onpromotion,transactions,city,state,type,cluster,Local_holiday_type,is_Local_holiday,Regional_holiday_type,is_Regional_holiday,National_holiday_type,is_National_holiday,oil_price_interpolated,earthquake_day,earthquake_impact,wage_day,family
0,2013-03-15,37,1.00,0.00,1492.00,Cuenca,Azuay,D,2,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,93.49,no,no,yes,AUTOMOTIVE
1,2013-03-15,38,2.00,0.00,1624.00,Loja,Loja,D,4,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,93.49,no,no,yes,AUTOMOTIVE
2,2013-03-15,39,4.00,0.00,1284.00,Cuenca,Azuay,B,6,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,93.49,no,no,yes,AUTOMOTIVE
3,2013-03-15,4,0.00,0.00,1428.00,Quito,Pichincha,D,9,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,93.49,no,no,yes,AUTOMOTIVE
4,2013-03-15,40,1.00,0.00,996.00,Machala,El Oro,C,3,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,93.49,no,no,yes,AUTOMOTIVE
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90931,2017-08-15,54,8.00,0.00,802.00,El Carmen,Manabi,C,3,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,47.57,no,no,yes,AUTOMOTIVE
90932,2017-08-15,6,7.00,0.00,1589.00,Quito,Pichincha,D,13,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,47.57,no,no,yes,AUTOMOTIVE
90933,2017-08-15,7,5.00,0.00,1780.00,Quito,Pichincha,D,8,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,47.57,no,no,yes,AUTOMOTIVE
90934,2017-08-15,8,4.00,0.00,2621.00,Quito,Pichincha,D,8,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,no_holiday,47.57,no,no,yes,AUTOMOTIVE
